In [ ]:
#pip install kfp==1.4.0 kfp-server-api==1.2.0

In [ ]:
import os, json
import ssl
import tempfile
import kfp

In [ ]:
URL = os.getenv("DKUBE_URL")  # DKube UI URL
token = os.environ.get("DKUBE_USER_ACCESS_TOKEN")  # DKube OAuth token from UI
ssl.CERT_REQUIRED = ssl.CERT_NONE      # workaround if above URL is not signed by a trusted CA

client = kfp.Client(host=URL + "/pipeline", client_id="1", existing_token=token)

In [ ]:
@kfp.dsl.pipeline(name="test", description="pipeline with 1 echo component")
def test_pipeline():
    kfp.dsl.ContainerOp("echo", image="bash", command=["echo", "hello"])

In [ ]:
# compile
f = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
kfp.compiler.Compiler().compile(test_pipeline, f.name)

In [ ]:
# upload
pipeline = client.upload_pipeline(f.name, pipeline_name="test pipeline")

In [ ]:
#create experiment to group pipeline runs
experiment = client.create_experiment(name="training")

In [ ]:
# run
run = client.run_pipeline(experiment.id, "test pipeline run", pipeline_id=pipeline.id, params={})
# wait for completion
r=client.wait_for_run_completion(run.id, 60 * 20) 

In [ ]:
#Upload new version of a pipeline
pipeline_id=client.get_pipeline_id("test pipeline")
new_pipline=client.upload_pipeline_version(f.name, 'Another version', pipeline_id=pipeline.id)

In [ ]:
# delete pipeline
client.delete_pipeline(pipeline.id) 